# **NLP com Flair**
Este tutorial é baseado no livro "Natural Language Processing with Flair".

# 6 - Modelos de Classificação

No notebook anterior treinamos um modelo de word embedding. O word embedding é útil em tarefas como NER e PoS, pois
nosso objetivo é justamente taggear palavras (tokens). Isso não é o caso no problema de classificação.

Por exemplo, suponho que temos um texto onde um juiz dá uma sentença. O objetivo é então criar um
modelo que identifica se a decisão foi favorável ou desfavorável. Esse tipo de coisa pode ser feita
usando ferramentas como Regex. Porém, esse tipo método é menos robusto caso o texto não seja tão padronizado.
No caso dos textos de sentenças, pode ser que o juiz sempre use algo como "julgo a sentença procedente". Porém, podem
haver variações, como "julgo a sentença à favor do réu", ou alguma varição não prevista, até mesmo um erro ortográfico.

A forma de lidar com esse tipo de problema envolve criar um embedding para a sentença/documento
como um todo, e não para cada palavra. Assim, queremos avaliar se a sentença
indica decisão positiva ou negativa.

Já falamos brevemente de document embeddings no primeiro notebook.
Um tipo bem simples é o chamado `DocumentPoolEmbeddings`, que nada mais é que uma média
dos word embeddings.

Outro é o `DocumentRNNEmbeddings`, que usa redes RNN. Porém, vamos focar no
`TransformerDocumentEmbeddings`, pois estes são os que costumam apresentar melhor desempenho.

## Importando TransformerDocumentEmbedding

Utilizaremos o modelo e dados do [legalnlp](https://github.com/felipemaiapolo/legalnlp). Baixe os dados desse repositório, que se encontram em `demo/data_base.csv`.

Vamos começar importando o embedding. Vamos usar o modelo BERTikal, que foi treinado em dados jurídicos.


Primeiro, rode o código abaixo caso não tenha os pacotes instalados. 

In [18]:
# !pip install legalnlp
# !pip install transformers
# !pip install flair
# !pip install pandas
# !pip install scikit-learn

# from legalnlp.get_premodel import *

# get_premodel('bert')
# !mkdir models
# !mv BERTikal ./models/

In [2]:
from legalnlp.clean_functions import *
from flair.embeddings import TransformerDocumentEmbeddings
from flair.data import Sentence

document_embeddings = TransformerDocumentEmbeddings('./models/BERTikal/')


# Precisamos ajustar esses parametros para evitar que o numero de tokens ultrapasse o limite
document_embeddings.tokenizer.model_max_length = 512
document_embeddings.truncate = True

/home/davibarreira/miniconda3/envs/nlp/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## GPU ou CPU?

Se tiver uma GPU, basta não roda o código abaixo, pois ele irá mudar para rodar na cpu.

In [3]:
# import flair, torch
# flair.device = torch.device('cpu') 

## Criando Corpus

In [4]:
import pandas as pd
from sklearn.utils import shuffle
import numpy as np


df = pd.read_csv('./data/data_base.csv')
df.drop(columns='Unnamed: 0', inplace=True)
df.dropna(inplace=True)
df = shuffle(df, random_state=7).reset_index(drop=True)

In [15]:
n_val   = int(int(df.shape[0]*0.7)* 0.1)
n_train = int(df.shape[0]*0.7-n_val)
n_test  = df.shape[0] - n_val - n_train

train = df.iloc[0:n_train].copy()
dev   = df.iloc[n_train: n_train + n_val].copy()
test  = df.iloc[n_train + n_val:].copy()

In [17]:
train.to_csv('./data/train.csv',index=False, sep='\t')
test.to_csv('./data/test.csv',index=False, sep='\t')
dev.to_csv('./data/dev.csv',index=False, sep='\t')

# Funcoes do legalnlp que limpa o texto
train['text'] = train.text.apply(clean_bert)
test['text'] = test.text.apply(clean_bert)
dev['text'] = dev.text.apply(clean_bert)

In [19]:
from flair.data import Corpus
from flair.datasets import CSVClassificationCorpus

# this is the folder in which train, test and dev files reside
data_folder = './data'

# column format indicating which columns hold the text and label(s)
columns = {0: 'text', 1: 'label'}

# load corpus containing training, test and dev data and if CSV has a header, you can skip it

lbl_type = 'label'
corpus: Corpus = CSVClassificationCorpus(data_folder,
                                         columns,
                                         skip_header=True,
                                         delimiter='\t',
                                         label_type=lbl_type)

2022-10-21 18:28:27,887 Reading data from data
2022-10-21 18:28:27,888 Train: data/train.csv
2022-10-21 18:28:27,889 Dev: data/dev.csv
2022-10-21 18:28:27,890 Test: data/test.csv


In [20]:
label_dict = corpus.make_label_dictionary(label_type=lbl_type)

2022-10-21 18:28:28,718 Computing label dictionary. Progress:


4063it [00:01, 3940.81it/s]

2022-10-21 18:28:29,757 Dictionary created for label 'label' with 4 values: H:Arquivado (seen 1905 times), H:Ativo (seen 1844 times), H:Suspenso (seen 314 times)


In [21]:
from flair.models import TextClassifier
from flair.trainers import ModelTrainer


classifier = TextClassifier(document_embeddings,
                            label_dictionary=label_dict,
                            label_type=lbl_type)

trainer = ModelTrainer(classifier, corpus)
trainer.fine_tune('classifier',
              learning_rate = 5e-05,
              mini_batch_size = 32,
              max_epochs = 4)

## Fazendo Previsões

Depois do modelo treinado, podemos usar o modelo para fazer previsões. Note que no Flair, a previsão
fica atrelada à sua sentença. Veja abaixo como usar.

In [ ]:
classifier = TextClassifier.load('classifier/final-model.pt')

In [ ]:
pred = Sentence(df.text.iloc[-1])
classifier.predict(pred)
pred